# CACD: VGG-16

In [1]:
import keras
import os, shutil
from keras import models, layers, losses, optimizers, metrics
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
%matplotlib inline

train_dir = 'data\\CACD\\train'
test_dir = 'data\\CACD\\test'

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(150, 150), 
    batch_size=64
)

test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(150, 150), 
    batch_size=64
)

C:\Users\HP\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 145482 images belonging to 49 classes.
Found 15893 images belonging to 49 classes.


# Defining custom metrix

In [2]:
def five_claas_diff(y_true, y_pred):
    return K.cast(K.abs(K.argmax(y_true, axis=-1) - K.argmax(y_pred, axis=-1)) <= 5,
                  K.floatx())

def ten_claas_diff(y_true, y_pred):
    return K.cast(K.abs(K.argmax(y_true, axis=-1) - K.argmax(y_pred, axis=-1)) <= 10,
                  K.floatx())


In [4]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))
                 
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [6]:
model = models.Sequential()

model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(49, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dense_2 (Dense)              (None, 49)                25137     
Total params: 18,934,641
Trainable params: 4,219,953
Non-trainable params: 14,714,688
_________________________________________________________________


In [7]:
conv_base.trainable = False
print('This is the number of trainable weights before freezing the conv base:', len(model.trainable_weights))

model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy, five_claas_diff, ten_claas_diff]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=2275, 
                              epochs=2, 
                              validation_data=test_generator,
                              validation_steps=250)

This is the number of trainable weights before freezing the conv base: 4
Epoch 1/2
2275/2275 [==============================] - 581s 255ms/step - loss: 3.7012 - categorical_accuracy: 0.0404 - five_claas_diff: 0.3986 - ten_claas_diff: 0.6524 - val_loss: 3.6319 - val_categorical_accuracy: 0.0474 - val_five_claas_diff: 0.4447 - val_ten_claas_diff: 0.6979
Epoch 2/2
2275/2275 [==============================] - 346s 152ms/step - loss: 3.5812 - categorical_accuracy: 0.0563 - five_claas_diff: 0.4466 - ten_claas_diff: 0.6995 - val_loss: 3.5997 - val_categorical_accuracy: 0.0464 - val_five_claas_diff: 0.4399 - val_ten_claas_diff: 0.6862


In [8]:
from keras.models import load_model
try:
    model = load_model('checkpoints\\cacd_vgg_16.h5', custom_objects={'five_claas_diff': five_claas_diff, 'ten_claas_diff':ten_claas_diff})
except:
    pass

# Fine tuning

In [ ]:
conv_base.trainable = True
# set_trainable = False
# for layer in conv_base.layers:
#     if layer.name == 'block5_conv1':
#         set_trainable = True
#     if set_trainable:
#         layer.trainable = True
#     else:
#         layer.trainable = False  

model.compile(
    optimizer=optimizers.RMSprop(lr=1e-4), 
    loss=losses.categorical_crossentropy, 
    metrics=[metrics.categorical_accuracy, five_claas_diff, ten_claas_diff]
)

history = model.fit_generator(train_generator, 
                              steps_per_epoch=2275, 
                              epochs=30, 
                              validation_data=test_generator, 
                              callbacks=[
                                  keras.callbacks.ModelCheckpoint('checkpoints\\cacd_vgg_16.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1),
                              ],
                              validation_steps=250)

Epoch 1/30
2275/2275 [==============================] - 918s 404ms/step - loss: 3.5077 - categorical_accuracy: 0.0505 - five_claas_diff: 0.4931 - ten_claas_diff: 0.7583 - val_loss: 3.3441 - val_categorical_accuracy: 0.0636 - val_five_claas_diff: 0.5949 - val_ten_claas_diff: 0.8564
Epoch 2/30
2275/2275 [==============================] - 913s 401ms/step - loss: 3.2997 - categorical_accuracy: 0.0675 - five_claas_diff: 0.6115 - ten_claas_diff: 0.8633 - val_loss: 3.2591 - val_categorical_accuracy: 0.0706 - val_five_claas_diff: 0.6376 - val_ten_claas_diff: 0.8708
Epoch 3/30
2275/2275 [==============================] - 913s 401ms/step - loss: 3.1959 - categorical_accuracy: 0.0779 - five_claas_diff: 0.6677 - ten_claas_diff: 0.8918 - val_loss: 3.2335 - val_categorical_accuracy: 0.0732 - val_five_claas_diff: 0.6546 - val_ten_claas_diff: 0.8875
Epoch 4/30
2275/2275 [==============================] - 913s 401ms/step - loss: 3.1035 - categorical_accuracy: 0.0863 - five_claas_diff: 0.7105 - ten_claa

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['categorical_accuracy']
val_acc = history.history['val_categorical_accuracy']
val_5_acc = history.history['val_five_claas_diff']
val_10_acc = history.history['val_ten_claas_diff']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.plot(epochs, val_5_acc, 'r', label='5 class val acc')
plt.plot(epochs, val_10_acc, 'g', label='10 class val acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()
plt.plot(epochs, loss, 'bo', label='Training losses')
plt.plot(epochs, val_loss, 'b', label='Validation losses')
plt.title('Training and validation loss')
plt.legend()

plt.show()